**Aisha Muhammad Nawaz  L200921**  
PySpark QUIZ # 2
8A BSCS MMD  
19th March 2024

Instructions:
Sentimental Analysis on Tweets
3 files given: tweets,negative,positive
Output count and categorized tweets in two files, ignore words whose length is less than 4


In [7]:
# #Running on Colab
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 121752 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u402-ga-2ubuntu1~22.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u402-ga-2ubuntu1~22

In [6]:
!sudo apt update


Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [770 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,604 kB]
Get:13 http://archive.ubuntu.c

In [8]:
#  Import the libraries we will need
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

# Create Spark session and ContextRun PySpark.
# create the session
conf = SparkConf().set("spark.ui.port","4050")
# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.appName("DataFrame").config('spark.ui.port', '4050').getOrCreate()
spark

In [24]:
tweets = sc.parallelize(['Wow! The weather is great today','This instructor hates students','I was not this bad at maths before','She sucks at lying'])
negative=list(sc.parallelize(['bad','hates','sucks']).collect())
positive=list(sc.parallelize(['great']).collect())

def findCat(line,negative,positive):
  nC=0
  nP=0
  for w in line:
    if(len(w)>4):
      if(w in negative):
        nC=nC+1
      elif(w in positive):
        nP=nP+1
  if(nP>nC):
    return 'Positive'
  else:
    return 'Negative'
tweetsCat=tweets.map(lambda x: (x,findCat(x.split(' '),negative,positive)))
tweetsCount=tweetsCat.map(lambda x: (x[1],x[0])).countByKey()

tweetsCat.saveAsTextFile('Category3.txt')
sc.parallelize([(s,c) for s,c in tweetsCount.items()]).saveAsTextFile('Count3.txt')

In [26]:
tweets = sc.parallelize(['Wow! The weather is great today', 'This instructor hates students', 'I was not this bad at maths before', 'She sucks at lying'])
negative = sc.parallelize(['bad', 'hates', 'sucks']).collect()
positive = sc.parallelize(['great']).collect()

def findCat(line, negative, positive):
    nC = 0
    nP = 0
    for w in line:
        w = w.lower().strip('.,?!')  # Convert to lowercase and remove punctuation
        if len(w) > 4:
            if w in negative:
                nC += 1
            elif w in positive:
                nP += 1
    if nP > nC:
        return 'Positive'
    else:
        return 'Negative'

tweetsCat = tweets.map(lambda x: (x, findCat(x.lower().split(), negative, positive)))
tweetsCount = tweetsCat.map(lambda x: (x[1], 1)).reduceByKey(lambda x,y:x+y)

tweetsCat.saveAsTextFile('Category5.txt')
tweetsCount.saveAsTextFile('Count5.txt')

